# Materials Modelling

```{admonition} John Stewart Bell
:class: tip
Theoretical physicists live in a classical world, looking out into a quantum-mechanical world.
```

<iframe class="speakerdeck-iframe" frameborder="0" src="https://speakerdeck.com/player/db5df387845c4802b0bbae449095899c" title="Machine Learning for Materials (Lecture 2)" allowfullscreen="true" style="border: 0px; background-clip: padding-box; background-color: rgba(0, 0, 0, 0.1); margin: 0px; padding: 0px; border-radius: 6px; box-shadow: rgba(0, 0, 0, 0.2) 0px 5px 40px; width: 100%; height: auto; aspect-ratio: 560 / 420;" data-ratio="1.3333333333333333"></iframe>

[Lecture slides](https://speakerdeck.com/aronwalsh/mlformaterials-lecture2-modelling)

## ⚡️ Crystal electrostatics

Hello again!

This activity aims to build your understanding of using Python for materials modelling. It will require you to think back to undergraduate lectures on crystallography and materials chemistry. You are encouraged to look up concepts that you are not familiar with.

We will use the computational materials science package `pymatgen` (https://pymatgen.org). Many Python packages have `py` at the start of their name. This one is useful to manipulate materials through its application programming interface (API). It also allows you to efficiently access and analyse the crystal structures and calculated properties that are available on [Materials Project](https://materialsproject.org).

### Theoretical background

The classical electrostatic interaction between point charges is described by Coulomb's law. For two interacting ions with charge $q$ and distance $r$ in a medium of dielectric constant $\epsilon_0$, the electrostatic energy is defined in the usual way:

$$
    E(r) = \frac{1}{4 \pi \epsilon_0}\frac{q^2}{r}
$$

Due to the repeating unit cells, the electrostatic energy of a crystal composed of ions is more complicated to compute. Here, the electrostatic energy is defined by a conditionally convergent series. The number of charges grows with the square of the radius used for the summation.

![image](./images/2_sum.png)

Ewald summation is a numerical technique to calculate the electrostatic potential energy of a crystal. The mathematical trick used is to split the electrostatic summation into real space and Fourier (reciprocal) space terms to ensure convergence at both short and long ranges. The technical background isn't important now, but is detailed over [here](https://en.wikipedia.org/wiki/Ewald_summation) if you are curious. The corresponding expression can be written as:

$$
E_{\text{Ewald}}(r) = \frac{1}{2} \sum_{i \neq j}^{N} q_i q_j \Bigg[\frac{\text{erf}(\alpha r_{ij})}{r_{ij}} + \frac{\text{erfc}(\alpha r_{ij})}{\kappa^2} \exp\Bigg(-\frac{\kappa^2 r_{ij}^2}{4}\Bigg)\Bigg] + \frac{1}{2} \sum_{i=1}^{N} q_i^2 \frac{\alpha}{\sqrt{\pi}}
$$

Luckily, we don't have to solve such an expression by hand. Within `pymatgen`, the `pymatgen.analysis.ewald` module contains the `EwaldSummation` class that enables us to compute the electrostatic energy for any given structure and will be used later.

In [ ]:
# Installation of libraries
!pip install pymatgen --quiet

In [1]:
# Import of modules
import pandas as pd  # Data manipulation with DataFrames
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Plotting
from pymatgen.core import Structure, Lattice  # Materials analysis for crystal structures
from pymatgen.analysis.ewald import EwaldSummation  # Ewald summation for charged systems
from pymatgen.analysis.structure_matcher import StructureMatcher  # Structure matching
import itertools  # Functions for efficient looping and iteration

## Create a virtual perovskite

The [`Structure` module](https://pymatgen.org/pymatgen.core.structure.html) contains the `Structure` class which can be used to represent the unit cell of a crystal.  Remember, this is the simplest repeat unit of a crystal that is defined by three unit cell lengths (**a**,**b**,**c**), three angles ($\alpha,\beta,\gamma$), and the fractional coordinates of the atoms that it contains.

Below, we will create a model of a cubic perovskite from its spacegroup, Pm3̅m (No. 221). For a cubic unit cell **a** = **b** = **c** and $\alpha = \beta = \gamma = 90 ^{\circ}$, so we only need to know the length of **a**.

We will now generate a structure for CsPbI<sub>3</sub> from a spacegroup using the class method `Structure.from_spacegroup()`.

![image](./images/2_CsPbI3.png)

This graphic was created using [VESTA](https://jp-minerals.org/vesta/en).

In [15]:
# Create a structure for cubic CsPbI3 using pymatgen
#
# We must define the spacegroup, unit cell (lattice) dimensions, 
# chemical species and their coordinates
#
CsPbI3 = Structure.from_spacegroup(
'Pm-3m', # Spacegroup for a cubic perovskite
Lattice.cubic(6.41), # Cubic spacing of 6.41 Å
['Pb2+', 'Cs+', 'I-'], # Unique species of the ABX3 compound
[[0.,0.,0.],[0.5,0.5,0.5],[0.,0.,0.5]] # Fractional atomic coordinates of each site
)

We can display information about the structure that we have created by simply printing it.

In [ ]:
# Print the structure details
print()

<details>
<summary> Code hint </summary>
You want to print your structure object CsPbI3
</details>

Within `pymatgen`, we can export crystal structures to a variety of file formats using the `to` method of the `Structure` class. Some of these include the Crystallographic Information Format (cif) and POSCAR (the standard file format for the density functional theory code VASP). A complete list of file formats can be found in [the docs](https://pymatgen.org/pymatgen.core.structure.html#pymatgen.core.structure.IStructure.to) for the `Structure` class.

The `filename` argument of the `to` method needs to be specified to save the file to disk. Only supplying the `fmt` argument will result in a string being returned.

In [ ]:
# Print the file to screen
print('\nThe CIF format for CsPbI3 is shown below:\n')
print(CsPbI3.to(fmt='cif'))

# Export the structure to a CIF
# CsPbI3.to(filename='CsPbI3.cif')

<details>
<summary> Code hint </summary>
You can export a .cif by uncommenting and running the lines above. These files can be opened in a visualiser of your choice (e.g. VESTA or Crystalmaker).
</details>

We can make an ionic approximation for CsPbI<sub>3</sub> and consider that the crystal is formed of Cs<sup>+</sup>, Pb<sup>2+</sup>, and I<sup>-</sup> ions. The crystal is charge neutral overall as the sum of cations (3$^+$) is balanced by the sum of anions (3$^-$). This model is a reasonable starting point, but it is worth remembering than bonding in real solids is more complicated than a simple classification of "ionic", "covalent" or "metallic" (e.g. see this [perspective](https://www.nature.com/articles/s41563-018-0165-7)).

In [ ]:
# Instantiate the EwaldSummation class with our perovskite structure
es = EwaldSummation(CsPbI)

# Print the computed Ewald Sum
print(f' The electrostatic energy of CsPbI3 is {es.total_energy:.2f} eV')

<details>
<summary> Code hint </summary>
Check that the chemical formula is correct.</details>

## Create a virtual double perovskite

A three-component ABX$_3$ perovskite isn't always enough to access the properties that we want. There are many ways to expand the crystal chemical space of perovskites. One approach is to double the unit cell from ABX$_3$ to A$_2$B$_2$X$_6$. We can replace one of the B sites by B' to form a A$_2$BB'X$_6$ compound. It turns out that this is the natural structure of the mineral elpasolite K$_2$NaAlF$_6$ (e.g. see a [review](https://www.sciencedirect.com/science/article/pii/S136970212030451X) on the topic).

![image](./images/2_Cs2AgBiI6.png)

A graphic of the double halide perovskite Cs<sub>2</sub>AgBiI<sub>6</sub>, where the octahedral B-site is occupied by two distint cations Ag and Bi.

### Partial site occupancies

Each species occupied a distinct atomic site in the original CsPbI<sub>3</sub> structure. This structure can be classified as ordered. However, many materials have more complex structures that include some form of disorder in how the species are distributed over crystallographic sites. 

For example, imagine a one-dimensional nanowire with frogs and foxes in a 1:1 ratio, where each site has a 50\% probabily of being occupied by 🐸 or 🦊. These may form local sequences such as (🐸🦊🐸🦊🐸🦊), (🐸🐸🦊🐸🦊🦊) and (🐸🐸🐸🦊🦊🦊).

This type of disorder in a crystal is termed partial site occupancy, where a mixture of atoms (and sometimes vacancies) occupy the same crystallographic site. It is common in substitutional alloys of the form A$_{1-x}$B$_x$. We can create structures with partial occupancies in `pymatgen` by specifying the occupancy of the atoms in the structure that we create. If Pb<sup>2+</sup> is replaced by an equivalent number of Ag<sup>+</sup> and Bi<sup>3+</sup> cations, then the charge balance of the crystal is maintained. In terms of charges:

$$
2 q(\mathrm{Pb}^{2+}) = 4^+ = q(\mathrm{Ag}^+) + q(\mathrm{Bi}^{3+})
$$

In [ ]:
# Replace Pb2+ ions with Ag+ and Bi3+ in CsPbI3

# Create a copy of the structure
Cs2AgBiI6 = CsPbI3.copy()

# Change the first ion (Pb2+) to 0.5Ag+, 0.5Bi3+
Cs2AgBiI6[0] = {'Ag+':0.5, 'Bi3+':0.5}

# Print the structure
print(Cs2AgBiI6)

To model the double perovskite structure, we have substituted both Ag and Bi onto the Pb site in CsPbI<sub>3</sub>.

The substitution onto the 5-atom unit cell has produced a statistically disordered structure due to Ag and Bi both occupying the same site with partial occupancies of 50%. To use this structure as an input for material modelling, we should make an ordered representation of the structure. This may be a set of ordered configurations that forms an ensemble to represent the disordered nature of the true material.

We can achieve this by creating a supercell. Expanding the unit cell twice in each direction (i.e. $2\textbf{a} \times 2\textbf{b} \times 2\textbf{c}$) produces a 40-atom supercell with 8 Pb sites. We could then distribute the 4 Ag and 4 Bi atoms across these 8 Pb sites. This approach introduces a new problem: which sites do we choose to substitute the Ag and Bi atoms? The choice of substitution would affect the structure and energy of the double perovskite, (⍰ ⍰ ⍰ ⍰ ⍰ ⍰ ⍰ ⍰)?

Let's create a supercell of our CsPbI<sub>3</sub> perovskite. We can do this using the `.make_supercell()` method of the `Structure` class.  Note this is an in-place operation, which means that the operation is applied to the existing `Structure` object and returns nothing.

In [ ]:
# Create a 2a x 2b x 2c supercell of the cubic CsPbI3 structure
supercell = CsPbI3.copy()
supercell.make_supercell(2) # or 2,2,2, i.e. doubled in each direction

# Print the supercell
print(supercell)

Before moving on, think about how you would try distribute the Ag<sup>+</sup> and Bi<sup>3+</sup> ions on the eight Pb<sup>2+</sup> sites, e.g. (🐸🦊🐸🦊🐸🦊🐸🦊) and (🐸🐸🐸🐸🦊🦊🦊🦊).

### Structure combinatorics

To model a disordered crystal at the nanoscale, one approach is to enumerate all of the different atomic configurations in a supercell. We should consider the disordered sites and calculate the possible permutations (arrangements) of the ions.

The number of permutations $\textit{P}$ for a crystallographic site with $N$ different atoms, which have multiplicities $m_i$, for $i = 1 ... N$ occupying the site can be calculated using the formula for a multi-set permutation:

$$
    P(m_1,m_2,...,m_N) = \frac{(m_1+m_2+...+m_N)!}{m_1!m_2!...m_N!}=\frac{(\sum_{i=1}^{N} m_i)!}{\prod_{i=1}^{N} k_i!}
$$

For our double perovskite structure, try to think of how many permutations of Ag and Bi are possible in the $2\textbf{a} \times 2\textbf{b} \times 2\textbf{c}$ supercell with 8 Pb sites.

###  Permutations of a string
For calculating permutations of an iterable object, we can use the `itertools.permutations()` function. A description of this function can be found in [`itertools` docs](https://docs.python.org/3/library/itertools.html#itertools.permutations). Note that the permutations produced using this function might not necessarily be unique.
To understand the idea, we will consider the permutations of the `string` '🐸🦊🦊🙀'.

Before you run the cell block below, think of the number of permutations of '🐸🦊🦊🙀'.

In [ ]:
# Generate permutations
permutations = itertools.permutations('🐸🦊🦊🙀') # This returns an itertools.permutation object

# Convert the above object to a list
perms_list = list(permutations)

# Print out the list
print(f"We have {len(perms_list)} permutations:")
print(p_list)

<details>
<summary> Code hint </summary>
Print the correct list in the last line.
</details>

From the above output, we can see that there are duplicate entries from the permutation of 🐸🦊🦊🙀. This happens as `itertools` considers elements in an iterable to be unique based of their position rather than their value. While this can be easy to spot in the example above, manually trying to filter out duplicate permutations is not an efficient use of our time.

Fortanately, we can reduce the permutations to a list of unique permutations quite simply using the `set` function.

In [ ]:
# Create a set for the permutations
perms_set = set(perms_list)
print(f"We have {len(perms_set)} unique permutations:")
print(perms_set)

We have removed the duplicate entries and returned the unique permutations of the string. Imagine how useful this is when dealing with complex structures containing hundreds of atoms and millions of permutations.

## 🚨 Exercise 2: Atomic permutations

```{admonition} Coding exercises
:class: note
The exercises are designed to apply what you have learned with room for creativity. It is fine to discuss solutions with your classmates, but the actual code should not be directly copied.

The completed notebooks are to be submitted at the end of class, but you can revist later, experiment with the code, and follow the further reading suggestions.
```

### Your details

In [ ]:
import numpy as np

# Insert your values
Name = "No Name" # Replace with your name
CID = 123446 # Replace with your College ID (as a numeric value with no leading 0s)

# Set a random seed using the CID value
CID = int(CID)
np.random.seed(CID)

# Print the message
print("This is the work of " + Name + " [CID: " + str(CID) + "]")

### Tasks

You have one task to complete involving the double perovskite Cs<sub>2</sub>AgBiI<sub>6</sub>:

1. Extend the code below to create unique permutations for Ag<sup>+</sup> and Bi<sup>3+</sup> ions in the perovskite supercell. How many unique permutations are there?

```python
# Create a 2a x 2b x 2c supercell of the cubic CsPbI3 structure
supercell = CsPbI3.copy()
supercell.make_supercell(2) # doubled in each direction

# Place Ag+ and Bi3+ on the Pb2+ atomic sites
for i in range(4):
  supercell[i] = 'Ag+'
for i in range(4, 8):
  supercell[i] = 'Bi3+'

# Define the set of these B-sites
b_species = supercell.species[0:8]
```

*Self-study (optional)*  

2. Generate structures corresponding to the unique permutations of Ag and Bi. This involves substituting the Pb sites in the crystal structure with Ag and Bi. Plot the distribution of electrostatic energies calculated using `EwaldSummation`.
  
3. Visualise the structures of the lowest energy configurations. Is there anything special about the Ag and Bi arrangement? Remember that in thermodynamic equilibrium, these configurations will be the most abundant (recall [Boltzmann statistics](https://en.wikipedia.org/wiki/Boltzmann_distribution)).

<details>
<summary> Task hint </summary>
One approach for task 2 involves populating a list of structures, e.g. starting from `Structures = []`.
</details>

```{admonition} Submission
:class: note
When your notebook is complete, click on the download icon on the top right, select `.pdf`, save the file and upload it to MyDepartment. If you are using Google Colab, you have to print to pdf.
```

In [ ]:
#Code block




In [ ]:
#Comment block




In [ ]:
#Code block




In [ ]:
#Comment block




## 🌊 Dive deeper

* _Level 1:_ If you are not comfortable with optimisation techniques such as gradient descent, read Chapter 2 of [Machine Learning Refined](https://github.com/jermwatt/machine_learning_refined#what-is-new-in-the-second-edition). Otherwise, skip to Chapter 5 on linear regression. 

* _Level 2:_ An introductory review on the Materials Project published in [APL Materials](https://aip.scitation.org/doi/10.1063/1.4812323).

* _Level 3:_ [icet](https://icet.materialsmodeling.org) is a powerful Python package for modelling disordered crystals based on the cluster expansion approach.